In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
np.set_printoptions(precision=3, suppress=True)
from wordcloud import WordCloud, STOPWORDS
import urllib.request
from PIL import Image
import re 
import emoji

from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils

In [2]:
# Tên các sheet trong file data.xlsx
name_sheet = ["ID","DATA","RATING"]

## Làm sạch dữ liệu cơ bản

### Sheet ID

In [3]:
# Tải dữ liệu sheet ID
data_ID = pd.read_excel('data.xlsx',sheet_name=name_sheet[0])
data_ID.head(10)

,item_id,shop_id,shop_location,name
0,6685849255,65589552,TP. Hồ Chí Minh,[Mã ELBAU1TR giảm 5% ] Điện Thoại Samsung Gala...
1,1236186398,1657913,Hà Nội,"Điện Thoại Nokia 101,Nokia 100, Nokia 105 Zin ..."
2,9764742653,453347243,TP. Hồ Chí Minh,Điện thoại TECNO POVA2 (4GB+64GB) - Hàng chính...
3,6602136609,70094786,TP. Hồ Chí Minh,Điện Thoại Siêu Dễ Thương Mini BM10 Thay Đổi G...
4,7297413303,65589552,TP. Hồ Chí Minh,[Mã ELBAU1TR giảm 5% ] Điện thoại Samsung Gala...
5,1605445818,1657913,Hà Nội,"Điện Thoại Nokia 101, Nokia 105 - Máy 2 Sim 2 ..."
6,3400182282,88201679,TP. Hồ Chí Minh,[Nhập ELAP500K giảm 8% đơn 500K] Apple iPhone ...
7,938839622,1657913,Hà Nội,Điện Thoại Nokia 1280 Zin Chính Hãng Màn Hình ...
8,4451525706,178181467,Nam Định,máy cũ giá rẻ oppo neo5 ram 1gb
9,8958237106,116554044,Hưng Yên,Nokia 6300 ★Chính hãng 100%★ Điện Thoại Phổ Th...


In [4]:
# Kiểm tra dữ liệu
data_ID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   item_id        3060 non-null   int64 
 1   shop_id        3060 non-null   int64 
 2   shop_location  3060 non-null   object
 3   name           3060 non-null   object
dtypes: int64(2), object(2)
memory usage: 95.8+ KB


In [5]:
# Kiểm tra giá trị dữ liệu
data_ID.nunique()

item_id          3058
shop_id           699
shop_location      44
name             2866
dtype: int64

In [6]:
# Cột item_id có 3060 hàng nhưng chỉ có 3058 giá trị => 2 hàng bị trùng
# Drop duplicate
data_ID = data_ID.drop_duplicates('item_id', keep='last')

In [7]:
# Cột name có các giá trị không cần thiết [thông tin khuyến mãi],
# giá trị \n, các emoji, nhiều khoảng trống tạo hàm để loại bỏ
def Khac_phuc(text):
    #Loại bỏ các từ trong ngoặc [] {} "" 【】 **
    text = re.sub("[\[\{\"\【\*].*?[\*\】\"\}\]]", "", text)
    # Thay _ bằng khoảng trắng
    text = re.sub("_", " ", text)
    # Thay ★ bằng khoảng trắng
    text = text.replace("★"," ")
    # Loại bỏ emoji
    text = emoji.replace_emoji(text, replace=" ")
    # Thay \n bằng khoảng trắng
    text = text.replace("\n"," ")
    text = text.replace("["," ")
    # Thay nhiều khoảng trống bắng 1 khoảng trống
    text.split()
    text1 = " ".join(text.split())
    return text1

In [8]:
#apply Hàm Khac phuc
data_ID['Tên'] = data_ID['name'].apply(Khac_phuc)

In [9]:
# Xóa cột name
data_ID = data_ID.drop(labels='name', axis=1)

In [10]:
# Đổi tên cột shop_location thành Tỉnh
data_ID = data_ID.rename(columns = {"shop_location":"Tỉnh"})

In [11]:
# Lưu lại
with pd.ExcelWriter('saved_file.xlsx',mode="w",) as writer:
    data_ID.to_excel(writer, index = False, sheet_name = 'ID')

### Sheet DATA

In [12]:
# Tải dữ liệu sheet DATA
data_DATA = pd.read_excel('data.xlsx',sheet_name=name_sheet[1])
data_DATA.head(10)

,item_id,shop_id,brand,sold_quantity,stock,discount_price,original_price,discount,liked_count,rating_star,number_of_ratings,product_image_link
0,6685849255,65589552,SAMSUNG,16347,2976,289000000000,349000000000,17%,4589,4.941775,2862,https://cf.shopee.vn/file/132b023a3114a559d723...
1,1236186398,1657913,nokia,20511,27855,9900000000,11000000000,10%,32530,4.647919,6298,https://cf.shopee.vn/file/3a7fe3513b59468fc5d3...
2,9764742653,453347243,TECNO,684,2256,289000000000,399000000000,28%,863,4.898734,316,https://cf.shopee.vn/file/a474488f5387c63543fe...
3,6602136609,70094786,NaN,13681,23,19500000000,24500000000,20%,1270,4.859685,3440,https://cf.shopee.vn/file/22ac0bf9b42272b91701...
4,7297413303,65589552,SAMSUNG,3270,229,439000000000,589000000000,25%,13413,4.921875,770,https://cf.shopee.vn/file/de40f4e661f2064711bf...
5,1605445818,1657913,NaN,6135,15414,13500000000,0,NaN,27367,4.594380,1638,https://cf.shopee.vn/file/834581faf97605e3839d...
6,3400182282,88201679,Apple,5342,503,1209000000000,2299000000000,47%,11637,4.907659,1399,https://cf.shopee.vn/file/e891e6f900bf8b1760a3...
7,938839622,1657913,nokia,10212,8290,15400000000,16500000000,7%,26581,4.666001,3004,https://cf.shopee.vn/file/331c288812971ce367ad...
8,4451525706,178181467,OPPO,1785,1430,45000000000,0,NaN,823,4.674682,707,https://cf.shopee.vn/file/09e9aa4af57127e01166...
9,8958237106,116554044,nokia,10573,29782,29900000000,0,NaN,221,4.698630,220,https://cf.shopee.vn/file/8bacefbbad71f03a9c8d...


In [13]:
# Kiểm tra dữ liệu
data_DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             3060 non-null   int64  
 1   shop_id             3060 non-null   int64  
 2   brand               2199 non-null   object 
 3   sold_quantity       3060 non-null   int64  
 4   stock               3060 non-null   int64  
 5   discount_price      3060 non-null   int64  
 6   original_price      3060 non-null   int64  
 7   discount            1329 non-null   object 
 8   liked_count         3060 non-null   int64  
 9   rating_star         3060 non-null   float64
 10  number_of_ratings   3060 non-null   int64  
 11  product_image_link  3060 non-null   object 
dtypes: float64(1), int64(8), object(3)
memory usage: 287.0+ KB


In [14]:
# Kiểm tra giá trị dữ liệu
data_DATA.nunique()

item_id               3058
shop_id                699
brand                   57
sold_quantity          914
stock                 1608
discount_price         970
original_price         468
discount                50
liked_count            913
rating_star            682
number_of_ratings      270
product_image_link    2989
dtype: int64

In [15]:
# Cột item_id có 3060 hàng nhưng chỉ có 3058 giá trị => 2 hàng bị trùng
# Drop duplicate
data_DATA = data_DATA.drop_duplicates('item_id', keep='last')

In [16]:
# Kiểm tra dữ liệu
data_DATA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3058 entries, 0 to 3059
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             3058 non-null   int64  
 1   shop_id             3058 non-null   int64  
 2   brand               2198 non-null   object 
 3   sold_quantity       3058 non-null   int64  
 4   stock               3058 non-null   int64  
 5   discount_price      3058 non-null   int64  
 6   original_price      3058 non-null   int64  
 7   discount            1329 non-null   object 
 8   liked_count         3058 non-null   int64  
 9   rating_star         3058 non-null   float64
 10  number_of_ratings   3058 non-null   int64  
 11  product_image_link  3058 non-null   object 
dtypes: float64(1), int64(8), object(3)
memory usage: 310.6+ KB


In [17]:
data_DATA.describe()

,item_id,shop_id,sold_quantity,stock,discount_price,original_price,liked_count,rating_star,number_of_ratings
count,3.058000e+03,3.058000e+03,3058.000000,3.058000e+03,3.058000e+03,3.058000e+03,3058.000000,3058.000000,3058.000000
mean,6.702710e+09,1.156114e+08,454.019948,8.574517e+04,1.619469e+11,1.175052e+11,1215.728581,4.770768,43.925114
std,4.082439e+09,1.215341e+08,1086.371098,1.643165e+06,2.543120e+11,2.729285e+11,6655.475278,0.301889,185.608370
min,4.242862e+06,1.103000e+04,2.000000,0.000000e+00,1.000000e+08,0.000000e+00,0.000000,0.000000,0.000000
25%,3.591593e+09,2.514343e+07,28.000000,2.800000e+01,2.650000e+10,0.000000e+00,11.000000,4.625000,6.000000
50%,6.207851e+09,6.919680e+07,97.000000,2.825000e+02,8.970000e+10,0.000000e+00,40.000000,4.869783,12.000000
75%,8.936892e+09,1.615503e+08,281.750000,9.026000e+03,1.999000e+11,1.650000e+11,294.250000,5.000000,28.000000
max,1.794023e+10,7.361472e+08,20511.000000,4.666662e+07,3.099000e+12,3.799000e+12,150471.000000,5.000000,6298.000000


In [18]:
# Đổi NaN của cột brand thành Other
data_DATA['brand'].fillna('Other', inplace=True)

In [19]:
data_DATA = data_DATA.reset_index()

In [20]:
# Cột original_price có Giá trị 0 và tại đó cột discount = Nan tức không có khuyến mãi vậy các ô có giá trị 0 
# ở cột original_price = các giá trị ở tương ứng ở cột discount_price

In [21]:
# Thay các giá trị 0 ở cột original_price thành giá trị tương ứng ở cột discount_price
Gia_goc = []
for i in range(data_DATA['original_price'].size):
    a = data_DATA['original_price'][i]
    b = data_DATA['discount_price'][i]
    if a != 0:
        Gia_goc.append(a)
    else:
        Gia_goc.append(b)        

In [22]:
# Thêm cột giá gốc mới vào
data_DATA['Giá gốc'] = Gia_goc

In [23]:
# Và giá gốc bất hợp lý nên chỉnh lại / 100000
data_DATA['Giá gốc'] = data_DATA['Giá gốc']/100000

In [24]:
# Đổi NaN của cột discount thành 0%
data_DATA['discount'].fillna('0%', inplace=True)

In [25]:
#Cột discount đang có giá trị str đổi thành kiểu float
def Thay_gia_tri(value):
    # Thay % bằng khoảng trắng
    value = value.replace("%"," ")
    value = float(value)
    value1 = value/100
    return value1

In [26]:
data_DATA['Khuyến mãi %'] = data_DATA['discount'].apply(Thay_gia_tri)

In [27]:
# Tính lại cột discount_price và đổi tên thành giá khuyến mãi
data_DATA['Giá khuyến mãi'] = data_DATA['Giá gốc'] - data_DATA['Khuyến mãi %']*data_DATA['Giá gốc']

In [28]:
# Giả sử các sp đều được bán bằng giá khuyến mãi
# Thêm cột doanh thu = sold_quantity * Giá sau khuyến mãi
data_DATA['Doanh thu'] = data_DATA['Giá khuyến mãi'] * data_DATA['sold_quantity']

In [29]:
# Xóa các cột không cần thiết
data_DATA = data_DATA.drop(labels=['index','discount_price','original_price','discount'], axis=1)

In [30]:
# Sắp xếp lại thứ tự các cột
data_DATA = data_DATA[['item_id', 'shop_id', 'brand', 'sold_quantity','stock','Giá gốc','Khuyến mãi %','Giá khuyến mãi','Doanh thu','liked_count','rating_star','number_of_ratings','product_image_link']]

In [31]:
data_DATA = data_DATA.rename(columns = {"sold_quantity":"Doanh số","stock":"Tồn kho","liked_count":"Lượt thích","rating_star":"Đánh giá","number_of_ratings":"Lượt đánh giá" })

In [32]:
with pd.ExcelWriter('saved_file.xlsx', mode="a", engine="openpyxl") as writer:
    data_DATA.to_excel(writer, index = False, sheet_name = 'DATA')

### Sheet RATING

In [33]:
# Tải dữ liệu sheet RATING
data_RATING = pd.read_excel('data.xlsx',sheet_name=name_sheet[2])
data_RATING.head(10)

,item_id,order_id,cmt_id,user_name,comment,rating_star
0,6685849255,112539818117052,9103860935,g*****4,Chất lượng sản phẩm:tốt\nTính năng nổi bật:tốt...,3
1,6685849255,111058415139684,8953928762,giacquanthu6,"Hàng Ok, đóng gói cẩn thận, nhìn chắc chắn đan...",5
2,6685849255,110883295199542,8924673029,molinguyen30011992,Mua cho ba\nĐặt màu đen giao màu xanh\nThôi cứ...,4
3,6685849255,109996550130678,8930604293,lntientrung,"Điện thoại rất tốt, pin trâu, đủ dùng cho các ...",5
4,6685849255,106117412399209,8547042012,w7uvt877vs,Chất lượng sản phẩm:Tốt\nTính năng nổi bật:Pin...,5
5,6685849255,105776892615824,8366454110,maphanduc,"Máy thiết kế đẹp, cấu hình ổn cho các tác vụ h...",5
6,6685849255,99335442520308,7903344004,w7uvt877vs,Hàng chính hãng. Sử dụng ok. Giá hợp lý. Pin t...,5
7,6685849255,102618239388473,7952985609,ngan_ha_meoheo,"Giao hàng nhanh, đóng gói cẩn thận. Hàng chính...",5
8,6685849255,95912956185836,7249663412,chaolata066,"Giao hàng nhanh. Đóng gói kỹ. Hàng chất lượng,...",5
9,6685849255,94239158525054,7243311882,candy_thuy,Shop giao hàng nhanh Bảo hành kĩ càng ạ đã v s...,5


In [34]:
# Kiểm tra dữ liệu
data_RATING.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61472 entries, 0 to 61471
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      61472 non-null  int64 
 1   order_id     61472 non-null  int64 
 2   cmt_id       61472 non-null  int64 
 3   user_name    59669 non-null  object
 4   comment      45664 non-null  object
 5   rating_star  61472 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 2.8+ MB


In [35]:
# Kiểm tra giá trị dữ liệu
data_RATING.nunique()

item_id         3056
order_id       52683
cmt_id         61351
user_name      39511
comment        35884
rating_star        5
dtype: int64

In [36]:
# Cột cmt_id có  61472 hàng nhưng chỉ có 61351 giá trị => có giá trị bị trùng
# Drop duplicate
data_RATING = data_RATING.drop_duplicates('cmt_id', keep='last')

In [37]:
# Đổi kiểu cột comment thành dạng str
data_RATING['comment'] = data_RATING['comment'].astype(str)

In [38]:
# Xử lý cột commet tương tự cột name
#apply Hàm Khac phuc
data_RATING['Bình luận'] = data_RATING['comment'].apply(Khac_phuc)

In [39]:
# Loại bỏ cột comment
data_RATING = data_RATING.drop(labels=['comment'], axis=1)

In [40]:
data_RATING = data_RATING.reset_index(drop=True)

## Làm sạch dữ liệu nâng cao (Phi cấu trúc)

In [41]:
Rank = pd.Series(['Cao nhất'] * data_RATING['rating_star'].size)
Rank[data_RATING['rating_star'] <= 3] = 'Thấp nhất'
data_RATING['Rank'] = Rank

In [42]:
def Lay_V_A_N(text):
    list_test = []
    str1 = " "
    x, y = ViPosTagger.postagging(ViTokenizer.tokenize(text))
    for i in range(len(x)):
        if (y[i] == 'A') or (y[i] == 'V') or (y[i] == 'N') or (y[i] == 'Np') or (y[i] == 'Ny') or (y[i] == 'Nc'):
            list_test.append(x[i])               
    return (str1.join(list_test))

In [43]:
data_cmt = data_RATING.copy()
#data_cmt.dropna(inplace=True, axis=0, how="any")
data_cmt_1 = data_cmt.reset_index()
data_cmt_1['VAN'] = data_cmt_1['Bình luận'].apply(Lay_V_A_N)

In [44]:
data_RATING = pd.merge(data_RATING, data_cmt_1[["cmt_id","VAN"]], on="cmt_id", how='left')

In [45]:
with pd.ExcelWriter('saved_file.xlsx', mode="a", engine="openpyxl") as writer:
    data_RATING.to_excel(writer, index = False, sheet_name = 'RATING')